<a href="https://colab.research.google.com/github/Jagoda222/HCML_2/blob/main/David_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install line_profiler
!pip install rdp
!pip install pandas
!pip install xgboost
!pip install scikit-learn
!pip install matplotlib
!pip install sys

ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys


In [ ]:

import sys
sys.path.append(r'IRonPy-master')
from iron import phi

In [ ]:

import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xgboost as xgb
import importlib
sys.path.append(r'ID-8A60')
import IDLoss
importlib.reload(IDLoss)
from IDLoss import id_loss
import sys
import time  # For timing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
sys.path.append(r'IRonPy-master')
from iron import phi

In [ ]:
df = pd.read_csv('sampled_data_7000.csv')
df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,gender,native-country,income,occupation
0,26,17,0,0,Private,9th,5,Never-married,Own-child,Black,Male,United-States,2515,Other-service
1,40,47,0,1977,Self-emp-not-inc,Some-college,10,Married-civ-spouse,Husband,White,Male,United-States,99999,Craft-repair
2,42,48,0,0,Private,Assoc-acdm,12,Divorced,Unmarried,White,Female,United-States,18045,Exec-managerial
3,40,25,0,0,State-gov,Some-college,10,Married-civ-spouse,Husband,White,Male,United-States,22412,Protective-serv
4,40,45,0,0,State-gov,Some-college,10,Never-married,Not-in-family,White,Female,United-States,30100,Adm-clerical


In [ ]:
# Drop education-num column (we hot-encode it with education column)
df = df.drop(columns=['education-num'])

#  List categorical columns to encode
categorical_cols = ['workclass', 'education', 'marital-status', 'relationship',
                    'race', 'gender', 'native-country', 'occupation']

#  One-hot encode categorical columns
df_train_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

## READY TO TRAIN!!

X = df_train_encoded.drop(columns=['income'])
y = df_train_encoded['income']


from sklearn.model_selection import train_test_split

# Bin continuous target into quartiles for stratification
y_binned = pd.qcut(y, q=4, labels=False)

# Stratified train/val split using binned labels (to balance the income label in train&split)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y_binned,
    random_state=42
)


In [ ]:
# Your protected attributes
prot_attrs = [col for col in X_train.columns if col.startswith('race_') or col.startswith('gender_')]

# Convert y to float64
y_train_float = np.array(y_train, dtype=np.float64)

# Initialize phi control object and compute phis
ph = phi.phi_control(pd.Series(y_train_float))
phis = np.array(phi.phi(pd.Series(y_train_float), ph), dtype=np.float64)

# Create the IDLoss objective function
loss = id_loss(ph, X_train, prot_attrs, approx=False)

# Initialize the model
model = xgb.XGBRegressor(
    objective=loss,
    n_estimators=30,
    max_depth=3,
    learning_rate=0.2,
    random_state=42,
    n_jobs=-1
)


# Initialize the model
model_2 = xgb.XGBRegressor(
    # objective=loss,
    n_estimators=30,
    max_depth=3,
    learning_rate=0.2,
    random_state=42,
    n_jobs=-1
)

# Fit the model
model.fit(X_train, y_train)
# Predict
y_pred = model.predict(X_val)



In [ ]:
pred_df = X_val.copy()
pred_df['pred_income'] = y_pred
pred_df['pred_income'] = pred_df['pred_income'].round(2)
pred_df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,pred_income
4109,40,51,0,1887,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,61087.398438
5295,40,26,0,0,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,51208.308594
3334,40,32,0,0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,52909.660156
6151,12,17,0,0,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,897.020020
2081,35,23,0,0,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,10570.549805


In [ ]:
mean_race_pred = {}
mean_gender_pred = {}


mean_race_pred['Privileged Race']=pred_df[pred_df['race_White']==True]['pred_income'].mean()
mean_race_pred['Not Privileged Race']=pred_df[pred_df['race_White']!=True]['pred_income'].mean()


mean_gender_pred['gender_Male']=pred_df[pred_df['gender_Male']==True]['pred_income'].mean()
mean_gender_pred['gender_Female']=pred_df[pred_df['gender_Male']!=True]['pred_income'].mean()

mean_race_pred

{'Privileged Race': np.float32(37774.57),
 'Not Privileged Race': np.float32(27568.35)}

In [ ]:
race_sp = pd.DataFrame(0.0, index=mean_race_pred.keys(), columns=mean_race_pred.keys())

for i in race_sp.index:
  for j in race_sp.columns:
    race_sp.loc[i, j] = mean_race_pred[i] - mean_race_pred[j]

race_sp

,Privileged Race,Not Privileged Race
Privileged Race,0.000000,10206.220703
Not Privileged Race,-10206.220703,0.000000


In [ ]:
gender_sp = pd.DataFrame(0.0, index=mean_gender_pred.keys(), columns=mean_gender_pred.keys())

for i in gender_sp.index:
  for j in gender_sp.columns:
    gender_sp.loc[i, j] = mean_gender_pred[i] - mean_gender_pred[j]

gender_sp

,gender_Male,gender_Female
gender_Male,0.000000,24157.302734
gender_Female,-24157.302734,0.000000


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("Evaluation Metrics:")
print(f"• MSE:  {mse:.4f}")
print(f"• MAE:  {mae:.4f}")
print(f"• R²:   {r2:.4f}")

# To view some predictions
print("\n Sample predictions:", y_pred[:10])


Evaluation Metrics:
• MSE:  462921120.0000
• MAE:  15887.0830
• R²:   0.3639

 Sample predictions: [61087.4    51208.31   52909.66     897.0234 10570.554  26520.229
 51208.31   75648.74    7741.2188 16390.586 ]


# Parameter Optimization

In [ ]:
# Your protected attributes
prot_attrs = [col for col in X_train.columns if col.startswith('race_') or col.startswith('gender_')]

# Convert y to float64
y_train_float = np.array(y_train, dtype=np.float64)

# Initialize phi control object and compute phis
ph = phi.phi_control(pd.Series(y_train_float))
phis = np.array(phi.phi(pd.Series(y_train_float), ph), dtype=np.float64)

# Create the IDLoss objective function
loss = id_loss(ph, X_train, prot_attrs, approx=False)


In [ ]:
opt_model = xgb.XGBRegressor(
    objective=loss,
    n_estimators=40,
    max_depth=5,
    learning_rate=0.15,
    random_state=42,
    n_jobs=-1
)

opt_model.fit(X_train, y_train)

,objective,<function id_...00220874D7600>
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [ ]:
parameters = {"n_estimators": [20, 25, 30, 35, 40],
              "max_depth": [1, 2, 3, 4, 5],
              "learning_rate": [0.1, 0.15, 0.2, 0.25, 0.3]}



In [ ]:
# grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1)
# grid.fit(X_train, y_train)
# grid.score(X_val,y_val)

In [ ]:
pred_df = X_val.copy()
pred_df['pred_income'] = opt_model.predict(X_val)
pred_df['pred_income'] = pred_df['pred_income'].round(2)
pred_df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,pred_income
4109,40,51,0,1887,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,66147.640625
5295,40,26,0,0,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,45419.578125
3334,40,32,0,0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,50117.738281
6151,12,17,0,0,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,1349.790039
2081,35,23,0,0,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,8128.359863


In [ ]:
mean_race_pred = {}
mean_gender_pred = {}


mean_race_pred['Privileged Race']=pred_df[pred_df['race_White']==True]['pred_income'].mean()
mean_race_pred['Not Privileged Race']=pred_df[pred_df['race_White']!=True]['pred_income'].mean()


mean_gender_pred['gender_Male']=pred_df[pred_df['gender_Male']==True]['pred_income'].mean()
mean_gender_pred['gender_Female']=pred_df[pred_df['gender_Male']!=True]['pred_income'].mean()

mean_race_pred

{'Privileged Race': np.float32(37755.703),
 'Not Privileged Race': np.float32(26944.615)}

In [ ]:
race_sp = pd.DataFrame(0.0, index=mean_race_pred.keys(), columns=mean_race_pred.keys())

for i in race_sp.index:
  for j in race_sp.columns:
    race_sp.loc[i, j] = mean_race_pred[i] - mean_race_pred[j]

race_sp

,Privileged Race,Not Privileged Race
Privileged Race,0.000000,10811.087891
Not Privileged Race,-10811.087891,0.000000


In [ ]:
gender_sp = pd.DataFrame(0.0, index=mean_gender_pred.keys(), columns=mean_gender_pred.keys())

for i in gender_sp.index:
  for j in gender_sp.columns:
    gender_sp.loc[i, j] = mean_gender_pred[i] - mean_gender_pred[j]

gender_sp

,gender_Male,gender_Female
gender_Male,0.000000,27330.824219
gender_Female,-27330.824219,0.000000


# Intersectional Groups

In [ ]:
mean_intersectional_pred = {}

mean_intersectional_pred['PrivRace_Male']=pred_df[(pred_df['race_White']==True) & (pred_df['gender_Male']==True)]['pred_income'].mean()
mean_intersectional_pred['NotPrivRace_Male']=pred_df[(pred_df['race_White']!=True) & (pred_df['gender_Male']==True)]['pred_income'].mean()

mean_intersectional_pred['PrivRace_Female']=pred_df[(pred_df['race_White']==True) & (pred_df['gender_Male']!=True)]['pred_income'].mean()
mean_intersectional_pred['NotPrivRace_Female']=pred_df[(pred_df['race_White']!=True) & (pred_df['gender_Male']!=True)]['pred_income'].mean()

mean_intersectional_pred

{'PrivRace_Male': np.float32(46260.16),
 'NotPrivRace_Male': np.float32(37958.402),
 'PrivRace_Female': np.float32(18782.514),
 'NotPrivRace_Female': np.float32(15259.258)}

In [ ]:
intersectional_sp = pd.DataFrame(0.0, index=mean_intersectional_pred.keys(), columns=mean_intersectional_pred.keys())

for i in intersectional_sp.index:
  for j in intersectional_sp.columns:
    intersectional_sp.loc[i, j] = mean_intersectional_pred[i] - mean_intersectional_pred[j]

intersectional_sp

,PrivRace_Male,NotPrivRace_Male,PrivRace_Female,NotPrivRace_Female
PrivRace_Male,0.000000,8301.757812,27477.646484,31000.902344
NotPrivRace_Male,-8301.757812,0.000000,19175.888672,22699.144531
PrivRace_Female,-27477.646484,-19175.888672,0.000000,3523.255859
NotPrivRace_Female,-31000.902344,-22699.144531,-3523.255859,0.000000
